# Challenger experiments

In [1]:
import os, mlflow
from dotenv import load_dotenv

load_dotenv(override=True)  # Carga las variables del archivo .env
EXPERIMENT_NAME = "/Users/estebangmzv@gmail.com/nyc-taxi-experiments"

mlflow.set_tracking_uri("databricks")
experiment = mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

## Parent experiments

### Dataset

In [2]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer

In [3]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [4]:
df_train = read_dataframe('../data/green_tripdata_2025-01.parquet')
df_test = read_dataframe('../data/green_tripdata_2025-02.parquet')

Feature Engineering + One Hot Encoding

In [5]:
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

In [6]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
categorical = ['PU_DO']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

X_val = preprocess(df_test, dv)

In [7]:
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

In [8]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
categorical = ['PU_DO']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

X_val = preprocess(df_test, dv)

Target

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_test[target].values

Definir los `dataset` como objetos de `mlflow` para poderlos trackear

In [10]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-02")

## Tunning de hiperparametros usando Optuna

In [16]:
import mlflow
import optuna
from optuna.samplers import TPESampler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import root_mean_squared_error
from mlflow.models.signature import infer_signature
import pathlib, pickle

### Tunning de hiperparametros 

In [12]:
# ------------------------------------------------------------
# Definir la función objetivo para Optuna (Random Forest)
#    - Los hiperparámetros se muestrean con Optuna.
#    - Entrena el modelo y calcula RMSE.
#    - Registra el experimento y los artefactos en MLflow.
# ------------------------------------------------------------
def objective_rf(trial: optuna.trial.Trial):
    # Hiperparámetros muestreados por Optuna
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "random_state": 42,
        "n_jobs": -1
    }

    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "random_forest")
        mlflow.log_params(params)

        # Crear y entrenar el modelo
        from sklearn.ensemble import RandomForestRegressor
        model = RandomForestRegressor(**params)
        model.fit(X_train, y_train)

        # Predicción y métrica en validación
        y_pred = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)

        # Registrar la métrica principal
        mlflow.log_metric("rmse", rmse)

        # Definir y registrar la "signature" para MLflow
        signature = infer_signature(X_val, y_pred)

        mlflow.sklearn.log_model(
            model,
            "model",
            input_example=X_val[:5],
            signature=signature
        )

    return rmse

In [13]:
# ------------------------------------------------------------
# Definir la función objetivo para Optuna (Gradient Boosting)
#    - Usa GradientBoostingRegressor de scikit-learn.
#    - Ajusta hiperparámetros con Optuna y registra cada run en MLflow.
# ------------------------------------------------------------
def objective_gb(trial: optuna.trial.Trial):
    # Hiperparámetros muestreados por Optuna
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "random_state": 42
    }

    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "gradient_boosting")
        mlflow.log_params(params)

        # Crear y entrenar el modelo
        from sklearn.ensemble import GradientBoostingRegressor
        model = GradientBoostingRegressor(**params)
        model.fit(X_train, y_train)

        # Predicción y métrica en validación
        y_pred = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)

        # Registrar la métrica principal
        mlflow.log_metric("rmse", rmse)

        # Definir y registrar la "signature" para MLflow
        signature = infer_signature(X_val, y_pred)

        mlflow.sklearn.log_model(
            model,
            "model",
            input_example=X_val[:5],
            signature=signature
        )

    return rmse


#### Random Forest

In [14]:
# ------------------------------------------------------------
# Función objetivo para Optuna
# ------------------------------------------------------------
def objective_rf(trial: optuna.trial.Trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "random_state": 42,
        "n_jobs": -1
    }

    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "random_forest")
        mlflow.log_params(params)

        model = RandomForestRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

        signature = infer_signature(X_val, y_pred)
        mlflow.sklearn.log_model(model, "model", input_example=X_val[:5], signature=signature)

    return rmse


# ------------------------------------------------------------
# Crear estudio Optuna
# ------------------------------------------------------------
sampler = TPESampler(seed=42)
study_rf = optuna.create_study(direction="minimize", sampler=sampler)

with mlflow.start_run(run_name="RandomForest Hyperparameter Optimization (Optuna)", nested=True):
    study_rf.optimize(objective_rf, n_trials=10)

    best_params = study_rf.best_params
    best_params["random_state"] = 42
    best_params["n_jobs"] = -1

    mlflow.log_params(best_params)
    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "random_forest",
        "feature_set_version": 1,
    })

    # Entrenar modelo final
    final_model = RandomForestRegressor(**best_params)
    final_model.fit(X_train, y_train)

    y_pred = final_model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # Guardar preprocesador
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

    # Registrar modelo en MLflow
    feature_names = dv.get_feature_names_out()
    input_example = pd.DataFrame(X_val[:5].toarray(), columns=feature_names)
    signature = infer_signature(input_example, y_val[:5])

    mlflow.sklearn.log_model(
        final_model,
        name="model",
        input_example=input_example,
        signature=signature
    )

[I 2025-10-28 19:38:10,733] A new study created in memory with name: no-name-1234ea96-6567-4905-b2b8-603291fee16d


🏃 View run debonair-fish-659 at: https://dbc-ca24b3b2-6e1a.cloud.databricks.com/ml/experiments/2891152338475381/runs/fa46843d556844cda980691b29b5c0a5
🧪 View experiment at: https://dbc-ca24b3b2-6e1a.cloud.databricks.com/ml/experiments/2891152338475381


[W 2025-10-28 19:38:15,940] Trial 0 failed with parameters: {'n_estimators': 250, 'max_depth': 29, 'min_samples_split': 8, 'min_samples_leaf': 3, 'max_features': 'sqrt'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\ITESOO\Datos_proyecto\nyc-taxi-predictions-2025\.venv\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\esteb\AppData\Local\Temp\ipykernel_25380\1197994593.py", line 20, in objective_rf
    model.fit(X_train, y_train)
  File "c:\ITESOO\Datos_proyecto\nyc-taxi-predictions-2025\.venv\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ITESOO\Datos_proyecto\nyc-taxi-predictions-2025\.venv\Lib\site-packages\sklearn\ensemble\_forest.py", line 486, in fit
    trees = Parallel(
            ^^^^^^^^^
  File "c:\

🏃 View run RandomForest Hyperparameter Optimization (Optuna) at: https://dbc-ca24b3b2-6e1a.cloud.databricks.com/ml/experiments/2891152338475381/runs/d695792bbabd4d24876c381d1d349ad2
🧪 View experiment at: https://dbc-ca24b3b2-6e1a.cloud.databricks.com/ml/experiments/2891152338475381


KeyboardInterrupt: 

#### Gradient Boost

In [17]:
# ------------------------------------------------------------
# Función objetivo para Optuna
# ------------------------------------------------------------
def objective_gb(trial: optuna.trial.Trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "random_state": 42
    }

    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "gradient_boosting")
        mlflow.log_params(params)

        model = GradientBoostingRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

        signature = infer_signature(X_val, y_pred)
        mlflow.sklearn.log_model(model, "model", input_example=X_val[:5], signature=signature)

    return rmse


# ------------------------------------------------------------
# Crear estudio Optuna
# ------------------------------------------------------------
sampler = TPESampler(seed=42)
study_gb = optuna.create_study(direction="minimize", sampler=sampler)

with mlflow.start_run(run_name="GradientBoosting Hyperparameter Optimization (Optuna)", nested=True):
    study_gb.optimize(objective_gb, n_trials=10)

    best_params = study_gb.best_params
    best_params["random_state"] = 42

    mlflow.log_params(best_params)
    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "gradient_boosting",
        "feature_set_version": 1,
    })

    # Entrenar modelo final
    final_model = GradientBoostingRegressor(**best_params)
    final_model.fit(X_train, y_train)

    y_pred = final_model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # Guardar preprocesador
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

    # Registrar modelo en MLflow
    feature_names = dv.get_feature_names_out()
    input_example = pd.DataFrame(X_val[:5].toarray(), columns=feature_names)
    signature = infer_signature(input_example, y_val[:5])

    mlflow.sklearn.log_model(
        final_model,
        name="model",
        input_example=input_example,
        signature=signature
    )

[I 2025-10-28 19:40:11,981] A new study created in memory with name: no-name-e6b1cf49-ac00-4479-acb6-be8b5b364946
2025/10/28 19:40:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/28 19:40:36 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


2025/10/28 19:40:36 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run intrigued-snail-676 at: https://dbc-ca24b3b2-6e1a.cloud.databricks.com/ml/experiments/2891152338475381/runs/28d737570b15410a8da20e367539fafc
🧪 View experiment at: https://dbc-ca24b3b2-6e1a.cloud.databricks.com/ml/experiments/2891152338475381


[I 2025-10-28 19:40:49,730] Trial 0 finished with value: 6.348716044936728 and parameters: {'n_estimators': 144, 'learning_rate': 0.2536999076681772, 'max_depth': 8, 'min_samples_split': 7, 'min_samples_leaf': 1, 'subsample': 0.662397808134481, 'max_features': 'log2'}. Best is trial 0 with value: 6.348716044936728.
2025/10/28 19:41:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/28 19:41:27 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run colorful-crane-239 at: https://dbc-ca24b3b2-6e1a.cloud.databricks.com/ml/experiments/2891152338475381/runs/c9a024ed3b6c40e2881a6f1567a1b80b
🧪 View experiment at: https://dbc-ca24b3b2-6e1a.cloud.databricks.com/ml/experiments/2891152338475381


...

[I 2025-10-28 19:44:56,702] Trial 8 finished with value: 5.443680534488849 and parameters: {'n_estimators': 51, 'learning_rate': 0.1601531217136121, 'max_depth': 8, 'min_samples_split': 8, 'min_samples_leaf': 4, 'subsample': 0.6296178606936361, 'max_features': None}. Best is trial 8 with value: 5.443680534488849.
2025/10/28 19:45:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/28 19:45:10 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run marvelous-sponge-623 at: https://dbc-ca24b3b2-6e1a.cloud.databricks.com/ml/experiments/2891152338475381/runs/5c39314389f74e9b8007c95a5419fd03
🧪 View experiment at: https://dbc-ca24b3b2-6e1a.cloud.databricks.com/ml/experiments/2891152338475381


[I 2025-10-28 19:45:15,451] Trial 9 finished with value: 8.918734592885501 and parameters: {'n_estimators': 206, 'learning_rate': 0.030816017044468066, 'max_depth': 2, 'min_samples_split': 4, 'min_samples_leaf': 2, 'subsample': 0.8918424713352255, 'max_features': 'log2'}. Best is trial 8 with value: 5.443680534488849.
2025/10/28 19:45:39 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


c:\ITESOO\Datos_proyecto\nyc-taxi-predictions-2025\.venv\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


🏃 View run GradientBoosting Hyperparameter Optimization (Optuna) at: https://dbc-ca24b3b2-6e1a.cloud.databricks.com/ml/experiments/2891152338475381/runs/38b00900ad764efba00517f8f7f60f15
🧪 View experiment at: https://dbc-ca24b3b2-6e1a.cloud.databricks.com/ml/experiments/2891152338475381


In [18]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="workspace.default.nyc-taxi-model"
)

Registered model 'workspace.default.nyc-taxi-model' already exists. Creating a new version of this model...


MlflowException: Not a proper runs:/ URI: runs://model. Runs URIs must be of the form 'runs:/<run_id>/run-relative/path/to/artifact'

In [ ]:
from mlflow import MlflowClient
import mlflow.pyfunc

In [ ]:
model_name = "workspace.default.nyc-taxi-model" 
versions = [2, 5, 6]  
metric_name = "validation-rmse"

In [ ]:
client = MlflowClient()

In [ ]:
versions_metadata = client.search_model_versions(f"name='{model_name}'")
ranked_versions = []

for version in versions_metadata:
    run_id = version.run_id
    try:
        metric_value = client.get_run(run_id).data.metrics[metric_name]
        ranked_versions.append([version._version, metric_value])
    except KeyError:
            print(f"Advertencia: La versión {version._version} no tiene la métrica '{metric_name}' registrada.")
    print(f"La version {version._version} del modelo nyc-taxi-model tiene un rmse de {metric_value}")

In [ ]:
ranked_versions = sorted(ranked_versions, key=lambda x: x[1])
ranked_versions

In [ ]:
champion = ranked_versions[0][0]
challenger = ranked_versions[1][0]

In [ ]:
all_versions = client.search_model_versions(f"name='{model_name}'")

print("\n--- Asignando Nuevos Alias ---")

# 4. Asignar el Alias 'champion'
client.set_registered_model_alias(
    name=model_name, 
    alias='champion', 
    version=champion
)
print(f"✅ Alias 'champion' asignado a la Versión {champion}.")

# 5. Asignar el Alias 'challenger'
client.set_registered_model_alias(
    name=model_name, 
    alias='challenger', 
    version=challenger
)
print(f"Alias 'challenger' asignado a la Versión {challenger}.")